In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Importing data**

In [ ]:
data_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
data_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
Id = data_test['Id']

We can see that some columns contain NaN's

# **Filling NaN's**

In [ ]:
data_train.info()

In [ ]:
for col in list(data_test.columns):
    if data_train[col].dtype=='object':
        if data_train[col].nunique() != data_test[col].nunique():
            print(col,":", data_train[col].nunique(), "uniques in train", data_test[col].nunique(), "uniques in test")
#due to different number of unique values we have to work with both datasets together

Due to different number of uniques in train and test set i will join train and test set to preprocess

Removing houses with very big price

In [ ]:
q97 = data_train.SalePrice.quantile(0.99)
q003 = data_train.SalePrice.quantile(0.003)

print('Removed outliers, price values in: [{:.2f}; {:.2f}]'.format(0, q97)) #best was 0.99
data_train = data_train[(data_train.SalePrice < q97)].reset_index()

In [ ]:
Target = data_train.SalePrice
data_train=data_train.drop('SalePrice',axis=1)
#saving target variable

In [ ]:
# Join the train and the test set
data = pd.concat([data_train, data_test], keys = ['train', 'test'])
data = data.drop(['index', 'Id'],axis = 1)
features = list(data.columns)
print('detected {:d} features'.format(len(features)))

Checking year features

In [ ]:
Check_years = data.columns[data.columns.str.contains(pat = 'Year|Yr')] 
data[Check_years.values].max().sort_values(ascending = False)

In [ ]:
Replace_year = data.loc[(data['GarageYrBlt'] > 2050), 'GarageYrBlt'].index.tolist()
data.loc[Replace_year, 'GarageYrBlt'] = data['GarageYrBlt'].mode()

# Visualizing NaNs

In [ ]:
Na_perc = []
for feature in features:
    percentage = len(data[feature][data[feature].isna() == True]) / len(data[feature])
    Na_perc.append(percentage)
Nas = pd.DataFrame({'feature' : features, 'Na_perc': Na_perc}).sort_values(by = 'Na_perc', ascending = False)
Nas = Nas[Nas.Na_perc > 0.01]
 
plt.figure(figsize=(7,7))
plt.title('NaN percentage')
plt.xlabel("Top features")
sns.barplot(x = Nas.Na_perc, y = Nas.feature, orient = 'h')
plt.show()

In [ ]:
num_features=[]
cat_features=[]
ord_features=[]
for feature in features:
    if str(data[feature].dtype) == 'object':
        data[feature] = data[feature].fillna('None')
        #data_test[feature] = data_test[feature].fillna('None')
        cat_features.append(feature)
        pass
    
    if  str(data[feature].dtype)=='float64':
        if data[feature].nunique() < 50:
            ord_features.append(feature)
            #data[feature] = data[feature].fillna(data[feature].mean())
        else:
            #data[feature] = data[feature].fillna(int(data[feature].mean()))
            num_features.append(feature)
        pass
    
    if str(data[feature].dtype) == 'int64':
        #data_test[feature] =data_test[feature]
        if data[feature].nunique() < 50:
            ord_features.append(feature)
            #data[feature] = data[feature].fillna(data[feature].mean())
        else:
            num_features.append(feature)
            #data[feature] = data[feature].fillna(int(data[feature].mean()))
        
        pass
    

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler
num_ord_data = data[num_features + ord_features].copy()
scaler = RobustScaler()
num_ord_data = scaler.fit_transform(num_ord_data)
imputer = KNNImputer(n_neighbors = 10, weights = 'uniform')
num_ord_data = imputer.fit_transform(num_ord_data)
num_ord_data_transformed = pd.DataFrame(num_ord_data, columns = num_features + ord_features)
#num_ord_data = scaler.inverse_transform(num_ord_data_transformed)
trans_back = scaler.inverse_transform(num_ord_data_transformed)
num_ord_data_transformed = pd.DataFrame(trans_back, columns = num_features + ord_features)

In [ ]:
data = data.reset_index()
for col in num_features + ord_features:
    data[col] = num_ord_data_transformed[col]

In [ ]:
print("Numeric features:", len(num_features))
print("Ordinal features:", len(ord_features))
print("Categorical features:", len(cat_features))
if len(num_features) + len(ord_features) + len(cat_features) != len(features):
    print("missing features")
else:
    print('no missing,', len(num_features) + len(ord_features) + len(cat_features), "features")

In [ ]:
max_list_cat = []
max_list_ord = []
idx_max_cat = []
idx_max_ord = []

for feature in cat_features:
    cat_vals = data[feature].value_counts(normalize=True)
    max_list_cat.append(cat_vals.max())
    idx_max_cat.append(cat_vals.idxmax())
    
max_zero_percentage_num = []
for feature in num_features:
    percentage = data[feature][data[feature] == 0].count() / len(data[feature])
    max_zero_percentage_num.append(percentage)

max_zero_percentage_ord = []
for feature in ord_features:
    ord_vals = data[feature].value_counts(normalize=True)
    max_list_ord.append(ord_vals.max())
    idx_max_ord.append(ord_vals.idxmax()) 

In [ ]:
zeroes_num = pd.DataFrame({'feature' : num_features, 'ratio' : max_zero_percentage_num}).sort_values(by = 'ratio', ascending=False)
vals_ord = pd.DataFrame({'feature' : ord_features, 'ratio' : max_list_ord, 'top_value' : idx_max_ord}).sort_values(by = 'ratio', ascending = False)
singl_el = pd.DataFrame({'feature' : cat_features, 'ratio' : max_list_cat, 'top_value' : idx_max_cat}).sort_values(by = 'ratio', ascending = False)

fig = plt.figure(figsize=(15,10))

plt.subplot(1, 3, 1)
plt.title('Max zero ratio in numeric data')
plt.xlabel("Top features")
sns.barplot(x = zeroes_num.ratio, y = zeroes_num.feature, orient = 'h')

plt.subplot(1, 3, 2)
plt.title('Max zero ratio in numeric data')
plt.xlabel("Top features")
sns.barplot(x = vals_ord.ratio, y = vals_ord.feature, orient = 'h')

plt.subplot(1, 3, 3)
plt.title('Max single element ratio')
plt.xlabel("Top features")
sns.barplot(x = singl_el.ratio, y = singl_el.feature, orient = 'h')
plt.subplots_adjust( bottom=None, right=None, top=None, wspace=1, hspace=None)
plt.show()
 

# Adiing new features

In [ ]:
data['TotalPorch'] = (data['ScreenPorch'] + data['EnclosedPorch'] + 
                         data['3SsnPorch'] + data['ScreenPorch'])

data['Rooms_kitchens'] = (data['TotRmsAbvGrd'] + data['BsmtFullBath'] + 
                             data['BsmtHalfBath'] + data['FullBath'] + 
                             data['HalfBath'])

data['Sqr_feet_per_room'] = ((data['1stFlrSF'] + 
                                 data['2ndFlrSF']) / data['TotRmsAbvGrd'])
data['lotfr_lotarea'] = (data['LotFrontage'] + data['LotArea']) / 2

data['TotalSF'] = data['TotalBsmtSF'] + data['1stFlrSF'] + data['2ndFlrSF']

data['cond_qual'] = (data['OverallCond'] * data['OverallQual'])
data['HouseAge'] = data['YrSold'] - data['YearBuilt'] + 1
#data['TotalPorch'].fillna(data['TotalPorch'].mean())
#data['Rooms_kitchens'].fillna(data['Rooms_kitchens'].mode())


num_features.append('TotalSF')
num_features.append('TotalPorch')
num_features.append('lotfr_lotarea')
num_features.append('HouseAge')
ord_features.append('Rooms_kitchens')
ord_features.append('cond_qual')


# Dealing with imbalanced features

In [ ]:
imb_num_features = zeroes_num.feature.head(3)
imb_ord_features = vals_ord[['feature', 'top_value']].head(5)
imb_cat_features = singl_el.feature[singl_el.top_value == 'None'].head(16)

In [ ]:
for feature in imb_num_features:
    data[feature][data[feature] != 0] = 0
    data[feature][data[feature] == 0] = 1
    num_features.remove(feature)
    cat_features.append(feature)
for feature in imb_ord_features.feature:
    data[feature][data[feature] != 0] = 0
    data[feature][data[feature] == 0] = 1
    ord_features.remove(feature)
    cat_features.append(feature)
for feature in imb_cat_features:
    data[feature][data[feature] != 'None'] = 'Present'

In [ ]:
print("Numeric features:", len(num_features))
print("Ordinal features:", len(ord_features))
print("Categorical features:", len(cat_features))
if len(num_features) + len(ord_features) + len(cat_features) != len(features):
    print("missing features")
else:
    print('no missing,', len(num_features) + len(ord_features) + len(cat_features), "features")

In [ ]:
for feature in cat_features:
    ratio = data[feature][data[feature] == 'None'].count() / len(data[feature])
    if ratio > 0.75: #best was 0.75
        cat_features.remove(feature)
        print('removed', feature)
for feature in ord_features:
    ratio = data[feature][data[feature] == 0].count() / len(data[feature])
    if ratio > 0.75: #best was 0.75
        ord_features.remove(feature)
        print('removed', feature)

I decided to use log of the data to prevent big values

In [ ]:
for col in num_features:
    data[col] = np.log(data[col] + 1)
for col in ord_features:
    data[col] = np.log(data[col] + 1)

In [ ]:
data_train = data[data.level_0 == 'train'].drop(['level_0', 'level_1'], axis = 1)
data_test = data[data.level_0 == 'test'].drop(['level_0', 'level_1'], axis = 1)

# EDA

Log of the price seems to be more normal

In [ ]:
Target.hist(bins = 20)
plt.show()
Target = np.log(Target + 1)
Target.hist(bins = 20)
plt.show()

In [ ]:
from matplotlib import cm

a = 8  # number of rows
b = 3  # number of columns
c = 1  # initialize plot counter

fig = plt.figure(figsize=(21,35))

for feature in num_features:
    plt.subplot(a, b, c)
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.scatter(data_train[feature], Target ,s = 2, label = feature, c=cm.cool(Target / Target.max()/2))
    c = c + 1  

We can see that some features have string correlation with the final price

In [ ]:
a = 5  # number of rows
b = 3  # number of columns
c = 1  # initialize plot counter

fig = plt.figure(figsize=(20,25))

for feature in ord_features:
    plt.subplot(a, b, c)
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.scatter(data_train[feature], Target ,s = 2, label = feature, c=cm.cool(Target / Target.max()/2))
    c = c + 1  

In [ ]:
data_train['SalePrice'] = Target

## Removing high collecalted variables

In [ ]:
fig, ax = plt.subplots(figsize=(11,9)) 
cmap = sns.diverging_palette(230, 20, as_cmap=True)
mask  = np.triu(np.ones_like(data_train[num_features].corr(), dtype=bool))
sns.heatmap(data_train[num_features].corr(),ax=ax,
            cmap = cmap, square=True, linewidths=.5, cbar_kws={"shrink": .5},
            mask=mask)

In [ ]:
num_features.remove('GarageYrBlt')
num_features.remove('1stFlrSF')
ord_features.remove('MoSold')
ord_features.remove('YrSold')

The graph below shows how our features are correlated with the price

In [ ]:
num_list = []
ord_list = []
for feature in num_features:
    num_list.append(data_train[feature].corr(data_train.SalePrice))

numcorrs = pd.DataFrame({'num_feature' : num_features, 'corr': num_list}).sort_values(by = 'corr', ascending = False)
for feature in ord_features:
    #print(feature,eda_data[feature].corr(eda_data.SalePrice))
    ord_list.append(data_train[feature].corr(data_train.SalePrice))
    #if abs(eda_data[feature].corr(eda_data.SalePrice)) < 0.1:
    #    ord_features.remove(feature)
ordcorrs = pd.DataFrame({'ord_feature' : ord_features, 'corr': ord_list}).sort_values(by = 'corr', ascending = False)
fig = plt.figure(figsize=(10,7))
plt.subplot(1,2,1)
plt.title('Correlation between numerical features and SalePrice')
sns.barplot(x = numcorrs['corr'], y = numcorrs['num_feature'], orient = 'h')
plt.subplot(1,2,2)
plt.title('Correlation between ordinal features and SalePrice')
sns.barplot(x = ordcorrs['corr'], y = ordcorrs['ord_feature'], orient = 'h')
plt.subplots_adjust( bottom=None, right=None, top=None, wspace=1, hspace=None)
    # compute correlation between sale price and other numeric features

# Removing the outliers

I will use Lasso model to remove the outliers by simply deleting rows with the largest residuals

In [ ]:
from sklearn.linear_model import Lasso
y_train = data_train['SalePrice']
data_train.drop('SalePrice', axis = 1)
data_train_dum = pd.get_dummies(data_train)
lasso = Lasso()
lasso.fit(data_train_dum, y_train)
outs = ((lasso.predict(data_train_dum) - y_train)**2).to_frame().sort_values(by = 'SalePrice', ascending = False)
bad_ind = outs[outs.SalePrice > 0.92].index.to_list() #best was 0.92
data_train = data_train.drop(bad_ind, axis = 0)
Target = Target.drop(bad_ind, axis = 0)

Getting some new variables

In [ ]:
data = pd.concat([data_train, data_test], keys = ['train', 'test']).reset_index()
data = data.drop('level_1', axis = 1)

I used the min max scaling to move the data into [0, 1]

In [ ]:
for col in num_features + ord_features:
    data[col] = (data[col] - data[col].min()) / (data[col].max() - data[col].min())
#for col in num_features + ord_features:
#    data[col] = (data[col] - data[col].mean()) / data[col].std()

In [ ]:
final_features=num_features + cat_features + ord_features
print("Amount of features:",len(final_features))
final_data = data[final_features]

#### There are some new features got from combinations

In [ ]:
from itertools import combinations

high_corr_num_features = ['GrLivArea', 'YearBuilt', 'YearRemodAdd', 'HouseAge']
high_corr_ord_features = ['OverallQual', 'Rooms_kitchens', 'GarageCars', 'FullBath', 'Fireplaces', 'TotRmsAbvGrd']

for c_1, c_2 in combinations(final_data[high_corr_num_features], 2):
    final_data['{0}x{1}'.format(c_1, c_2)] = final_data[c_1] * final_data[c_2]
    final_data['{0}+{1}'.format(c_1, c_2)] = (final_data[c_1] + final_data[c_2]) / 2 
for c_1, c_2 in combinations(final_data[high_corr_ord_features], 2):
    final_data['{0}x{1}'.format(c_1, c_2)] = final_data[c_1] * final_data[c_2]
    #final_data['{0}+{1}'.format(c_1, c_2)] = (final_data[c_1] + final_data[c_2])/2



# Final data

I used get_dummies method to create dummy vars for categorical columns

In [ ]:
final_data_dummies=pd.get_dummies(final_data, drop_first = False)
final_data_dummies['level']=data.level_0
final_data_dummies_train=final_data_dummies[final_data_dummies.level=='train']
final_data_dummies_test=final_data_dummies[final_data_dummies.level=='test']

In [ ]:
X = final_data_dummies_train.drop('level',axis=1)
y = Target
X_test = final_data_dummies_test.drop('level', axis = 1)

# Training Models and Tuning them with Hyperopt

There are some useful functions to measure the quality of the models

In [ ]:
def evaluate(base_model, tuned_model, X_train, y_train, test_features, test_labels, fit_params=None):
    base_model.fit(X_train, y_train)
    predictions = base_model.predict(test_features)
    print('Model Performance')
    print('MSE = {:0.4f}'.format(mean_squared_error(predictions, test_labels, squared = False)))
    
    base_mse = mean_squared_error(predictions, test_labels)
    if 'XGBRegressor' in str(tuned_model):
        tuned_model.fit(X_train, y_train, eval_set=[(X_train, y_train), (test_features, test_labels)], **fit_params)
    elif 'LGBMRegressor' in str(tuned_model):
        tuned_model.fit(X_train, y_train, eval_set=[(X_train, y_train), (test_features, test_labels)], **fit_params)
    else:
        tuned_model.fit(X_train, y_train, **fit_params)
    predictions = tuned_model.predict(test_features)
    print('Model Performance')
    print('MSE = {:0.4f}'.format(mean_squared_error(predictions, test_labels, squared = False)))
    tuned_mse = mean_squared_error(predictions, test_labels)
    print('Improvement of {:0.2f}%.'.format( 100 * (base_mse - tuned_mse) / base_mse))
    

#### Hyperopt tuner class 

In [ ]:
from hyperopt import hp, fmin, tpe, STATUS_OK, STATUS_FAIL, Trials

class HPOpt(object):

    def __init__(self, x_train, x_test, y_train, y_test):
        self.x_train = x_train
        self.x_test  = x_test
        self.y_train = y_train
        self.y_test  = y_test

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials

    def xgb_reg(self, para):
        reg = xgb.XGBRegressor(**para['reg_params'])
        return self.train_reg(reg, para)
    
    def rf_reg(self, para):
        reg = RandomForestRegressor(**para['reg_params'])
        return self.train_reg(reg, para)
    
    def lasso_reg(self, para):
        reg = linear_model.Lasso(**para['reg_params'])
        return self.train_reg(reg, para)
    
    def elnet_reg(self, para):
        reg = linear_model.ElasticNet(**para['reg_params'])
        return self.train_reg(reg, para)

    def lgb_reg(self, para):
        reg = lgb.LGBMRegressor(**para['reg_params'])
        return self.train_reg(reg, para)
    
    def svr_reg(self, para):
        reg = SVR(**para['reg_params'])
        return self.train_reg(reg, para)


    def train_reg(self, reg, para):
        if 'RandomForestRegressor'  in str(reg):
            reg.fit(self.x_train, self.y_train)
        elif 'Lasso' in str(reg):
            reg.fit(self.x_train, self.y_train)
        elif 'ElasticNet' in str(reg):
            reg.fit(self.x_train, self.y_train)
        elif 'SVR' in str(reg):
            reg.fit(self.x_train, self.y_train)
        else:   
            reg.fit(self.x_train, self.y_train,
                eval_set=[(self.x_train, self.y_train), (self.x_test, self.y_test)],
                **para['fit_params'])
        pred = reg.predict(self.x_test)
        loss = para['loss_func'](self.y_test, pred)
        return {'loss': loss, 'status': STATUS_OK}

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size = 0.3, random_state = 42)

### Lasso

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

lasso = linear_model.Lasso()

alpha = {'alpha': [x / 25000 for x in range(1, 50, 1)],
         'tol': [0.0000001], 
          'max_iter': [3000]}


lasso.fit(X_train, y_train)

In [ ]:
max_iter = range(100, 5000, 100)

lasso_reg_params = {'alpha':              hp.uniform('alpha', 0.00000001, 0.5),
                 'tol':                hp.uniform('tol', 0.0000001, 0.5),
                 'max_iter':           hp.choice('max_iter', max_iter),
                }
lasso_fit_params = {
                    'sample_weight' : None
}
obj = HPOpt(X_train, X_holdout, y_train, y_holdout)


#rf_fit_params = {
#    'sample_weight': None,
#}
lasso_para = dict()
lasso_para['reg_params'] = lasso_reg_params
lasso_para['fit_params'] = lasso_fit_params
lasso_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))

lasso_opt_para = obj.process(fn_name='lasso_reg', space=lasso_para, trials=Trials(), algo=tpe.suggest, max_evals=500)

In [ ]:
#best 0.0929 <- 0.1389

In [ ]:
lasso_opt_para
#{'alpha': 0.0003423653832332181, 'max_iter': 14, 'tol': 0.021055934172052676}

In [ ]:
lasso_reg_opt_params = {'alpha':               lasso_opt_para[0]['alpha'],
                        'tol':                 lasso_opt_para[0]['tol'],
                        'max_iter':            max_iter[lasso_opt_para[0]['max_iter']],
                        }
lasso_opt = linear_model.Lasso(**lasso_reg_opt_params)

In [ ]:
evaluate(lasso, lasso_opt, X_train, y_train, X_holdout, y_holdout, lasso_fit_params)

### XGB

In [ ]:
import xgboost as xgb
xg_reg = xgb.XGBRegressor(learning_rate = 0.1,
                          max_depth = 4, 
                          objective="reg:squarederror", 
                          n_estimators=700)
xg_reg.fit(X_train,y_train)

In [ ]:
obj = HPOpt(X_train, X_holdout, y_train, y_holdout)
booster = ['gbtree']
learning_rate = np.arange(0.01, 0.7, 0.05)
max_depth = range(5, 20, 1)
min_child_weight = range(1, 20, 1)
n_estimators = range(100, 5000, 100)

xgb_reg_params = {
    'booster':          hp.choice('booster',          booster),
    'learning_rate':    hp.choice('learning_rate',    learning_rate),
    'max_depth':        hp.choice('max_depth',        max_depth),
    'min_child_weight': hp.choice('min_child_weight', min_child_weight),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.0001, 1),
    'subsample':        hp.uniform('subsample', 0.0001, 1),
    'n_estimators':     hp.choice('n_estimators',     n_estimators)
}
xgb_fit_params = {
    'eval_metric': 'rmse',
    'early_stopping_rounds': 10,
    'verbose': False
}
xgb_para = dict()
xgb_para['reg_params'] = xgb_reg_params
xgb_para['fit_params'] = xgb_fit_params
xgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))

xgb_opt_para = obj.process(fn_name='xgb_reg', space=xgb_para, trials=Trials(), algo = tpe.suggest, max_evals=120)

In [ ]:
#best 0.1000 <- 0.1422

In [ ]:
xgb_reg_opt_params = {
    'booster':          'gbtree',
    'learning_rate':    learning_rate[xgb_opt_para[0]['learning_rate']],
    'max_depth':        max_depth[xgb_opt_para[0]['max_depth']],
    'min_child_weight': min_child_weight[xgb_opt_para[0]['min_child_weight']],
    'colsample_bytree': xgb_opt_para[0]['colsample_bytree'],
    'subsample':        xgb_opt_para[0]['subsample'],
    'n_estimators':     n_estimators[xgb_opt_para[0]['n_estimators']]
}
xgb_reg_opt = xgb.XGBRegressor(**xgb_reg_opt_params)
xgb_reg_opt_params

In [ ]:
evaluate(xg_reg, xgb_reg_opt, X_train, y_train, X_holdout, y_holdout, xgb_fit_params)

### Support Vector Machine

In [ ]:
from sklearn.svm import SVR

svr_reg = SVR()

In [ ]:
C = list(range(1, 100, 1))
epsilon = [x / 2000 for x in range(1, 50, 1)]
gamma = [x / 10000 for x in range(1, 50, 1)]
svr_reg_params = {
              'C' : hp.choice('C', C),
              'epsilon' : hp.uniform('epsilon', 0, 0.05),
              'gamma' :   hp.uniform('gamma', 0, 0.05)}

svr_fit_params = {
                    'sample_weight' : None
}
obj = HPOpt(X_train, X_holdout, y_train, y_holdout)



svr_para = dict()
svr_para['reg_params'] = svr_reg_params
svr_para['fit_params'] = svr_fit_params
svr_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))

svr_opt_para = obj.process(fn_name='svr_reg', space=svr_para, trials=Trials(), algo=tpe.suggest, max_evals=200)


In [ ]:
#best 0.092 <- 0.1163

In [ ]:
svr_opt_para

In [ ]:
svr_reg_opt_params = {  'kernel':        'rbf',
                        'C':              C[svr_opt_para[0]['C']],
                        'epsilon':         svr_opt_para[0]['epsilon'],
                        'gamma':         svr_opt_para[0]['gamma'],
                        }

svr_reg_opt = SVR(**svr_reg_opt_params)

In [ ]:
evaluate(svr_reg, svr_reg_opt, X_train, y_train, X_holdout, y_holdout, svr_fit_params)

### LGBM

In [ ]:
import lightgbm as lgb
lgb_reg = lgb.LGBMRegressor()
lgb_reg.fit(X_train, y_train)

In [ ]:
obj = HPOpt(X_train, X_holdout, y_train, y_holdout)
reg_lambda = [0.01, 0.05, 0.1, 0.2, 0.5, 1, 2]
learning_rate = np.arange(0.01, 0.5, 0.05)
reg_alpha = np.arange(0.01, 2, 0.05)
min_child_samples = range(1, 100, 1)
num_leaves = range(2, 200, 1)
subsample_freq = range(1, 200, 1)
max_depth = range(5, 20, 1)
min_child_weight = range(1, 20, 1)
n_estimators = range(100, 5000, 100)
max_bin = range(2, 700, 1)

LGBM_params = {'reg_lambda':                  hp.choice('reg_lambda', reg_lambda),
                      'reg_alpha':            hp.choice('reg_alpha', reg_alpha),
                      'min_child_samples':    hp.choice('min_child_samples', min_child_samples),
                      'subsample':            hp.uniform('subsample', 0.01, 1), # bagging_fraction
                      'subsample_freq':       hp.choice('subsample_freq', subsample_freq), # bagging_freq
                      'num_leaves':           hp.choice('num_leaves', num_leaves),
                      'max_depth':            hp.choice('max_depth', max_depth),
                      'max_bin':              hp.choice('max_bin', max_bin),
                      'learning_rate':        hp.choice('learning_rate', learning_rate),
                      'colsample_bytree':     hp.uniform('colsample_bytree', 0.01, 1)} # feature_fraction 
lgb_fit_params = {
    'eval_metric': 'rmse',
    'early_stopping_rounds': 10,
    'verbose': False
}
lgb_para = dict()
lgb_para['reg_params'] = LGBM_params
lgb_para['fit_params'] = lgb_fit_params
lgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))

lgb_opt_params = obj.process(fn_name='lgb_reg', space=lgb_para, trials=Trials(), algo=tpe.suggest, max_evals=300)

In [ ]:
#best 0.1048 <- 0.1191

In [ ]:
lgb_reg_params = {
    'reg_lambda':            reg_lambda[lgb_opt_params[0]['reg_lambda']],
    'reg_alpha':             reg_alpha[lgb_opt_params[0]['reg_alpha']],
    'min_child_samples':     min_child_samples[lgb_opt_params[0]['min_child_samples']],
    'subsample':             lgb_opt_params[0]['subsample'], 
    'subsample_freq':        subsample_freq[lgb_opt_params[0]['subsample_freq']], # 
    'num_leaves':            num_leaves[lgb_opt_params[0]['num_leaves']],
    'max_depth':             max_depth[lgb_opt_params[0]['max_depth']],
    'max_bin':               max_bin[lgb_opt_params[0]['max_bin']],
    'learning_rate':         learning_rate[lgb_opt_params[0]['learning_rate']],
    'colsample_bytree':      lgb_opt_params[0]['colsample_bytree'] #  
}

lgb_opt = lgb.LGBMRegressor(**lgb_reg_params)
lgb_opt.fit(X_train, y_train,
                eval_set=[(X_train, y_train), (X_holdout, y_holdout)],
                **lgb_para['fit_params'])
print(np.sqrt(mean_squared_error(lgb_opt.predict(X_holdout), y_holdout)))


In [ ]:
evaluate(lgb_reg, lgb_opt, X_train, y_train, X_holdout, y_holdout, lgb_fit_params)

### ElasticNet

In [ ]:
from sklearn.linear_model import ElasticNet
base_elnet = linear_model.ElasticNet()
max_iter = range(100, 5000, 100)

elnet_reg_params = {'alpha':              hp.uniform('alpha', 0.00000001, 0.5),
                    'tol':                hp.uniform('tol', 0.0000001, 0.5),
                    'max_iter':           hp.choice('max_iter', max_iter),
                    'l1_ratio':           hp.uniform('l1_ratio', 0.0000001, 0.5),
                }
elnet_fit_params = {
                    'sample_weight' : None
}
obj = HPOpt(X_train, X_holdout, y_train, y_holdout)


#rf_fit_params = {
#    'sample_weight': None,
#}
elnet_para = dict()
elnet_para['reg_params'] = elnet_reg_params
elnet_para['fit_params'] = elnet_fit_params
elnet_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))

elnet_opt_para = obj.process(fn_name='elnet_reg', space=elnet_para, trials=Trials(), algo=tpe.suggest, max_evals=500)


In [ ]:
#best 0.0932 <- 0.1163

In [ ]:
elnet_opt_para

In [ ]:
elnet_reg_opt_params = {'alpha':               elnet_opt_para[0]['alpha'],
                        'tol':                 elnet_opt_para[0]['tol'],
                        'max_iter':            max_iter[elnet_opt_para[0]['max_iter']],
                        'l1_ratio':            elnet_opt_para[0]['l1_ratio']
                        }
elnet_opt = linear_model.ElasticNet(**elnet_reg_opt_params)

In [ ]:
evaluate(base_elnet, elnet_opt, X_train, y_train, X_holdout, y_holdout, elnet_fit_params)

In [ ]:
print(np.sqrt(mean_squared_error((lasso_opt.predict(X_holdout) 
                                  + xgb_reg_opt.predict(X_holdout) 
                                  + lgb_opt.predict(X_holdout) 
                                  + elnet_opt.predict(X_holdout)
                                  + svr_reg_opt.predict(X_holdout)) / 5, y_holdout)))

In [ ]:
#best 0.0899 <- 0.11

# Averaging the predictions

I decided to combine the predictions of my models in order to prevent overfitting and to use advantages of all models. The idea is to make a weightened mean prediction and optimize it using Adam or SGD optimizers

In [ ]:
import torch
p = torch.tensor([lasso_opt.predict(X_holdout),
                  xgb_reg_opt.predict(X_holdout),
                  lgb_opt.predict(X_holdout),
                  elnet_opt.predict(X_holdout),
                  svr_reg_opt.predict(X_holdout)])
y = torch.tensor(np.array(y_holdout))
w = torch.tensor([[1.],[1.], [1.], [1.], [1.]], requires_grad = True)
def f(p, y, w):
    return torch.sqrt(torch.sum((y - torch.sum(w*p, dim = 0)/w.sum())**2)/len(y))

optimizer = torch.optim.Adam([w], lr = 1)
for i in range(5000):
    fun = f(p, y, w)
    fun.backward()
    optimizer.step()
    optimizer.zero_grad()
print('best score', f(p, y, w))
wf = np.array([[w.data[0]], [w.data[1]], [w.data[2]], [w.data[3]], [w.data[4]]])

In [ ]:
#best 0.0892 <- 0.1127

In [ ]:
print(wf)

Here we can see the difference between true and predicted values on a holdout set

In [ ]:
predictions = np.array([lasso_opt.predict(X_holdout),
                        xgb_reg_opt.predict(X_holdout),
                        lgb_opt.predict(X_holdout),
                        elnet_opt.predict(X_holdout),
                        svr_reg_opt.predict(X_holdout)])
plt.figure(figsize = (10, 9))
plt.plot([10.5, 11, 12, 13],[10.5, 11, 12, 13])
plt.xlabel('ensemble pred')
plt.ylabel('true values')
plt.scatter(np.sum((wf*predictions) / wf.sum(), axis = 0), y_holdout, c = 'red', alpha=0.5)
plt.show()

In [ ]:
pr = np.array([lasso_opt.predict(X_test),
               xgb_reg_opt.predict(X_test),
               lgb_opt.predict(X_test),
               elnet_opt.predict(X_test),
               svr_reg_opt.predict(X_test)])



pred = np.sum((wf*pr)/wf.sum(), axis = 0)

preds = pd.DataFrame({'Id' : Id, 'SalePrice': np.exp(pred)})
preds.to_csv('sumb.csv', index = False)
